<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/CNN_from_scratch_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are on CoLaB with torch version {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)} {e}\n>>>> please correct {type(e)} and reload your drive")
  COLAB  = False
def time_fmt(t: float = 123.918)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h} min: {m:>02} sec: {s:>05.2f}"
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f">>>> time formating\t........................\n>>>> time elapsed\t{time_fmt()}")

Mounted at /content/drive
>>>> You are on CoLaB with torch version 1.8.1+cu101
>>>> time formating	........................
>>>> time elapsed	hrs: 0 min: 02 sec: 03.00


In [8]:
#In this notebook we are going to implementing a simple Convolutional neural network from scratch using pytorch
#Demonstration using the famous MNIST images from Yann Lecun Paper will be applied.

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import time, sys, os


In [12]:
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):
    super(CNN, self).__init__()
    self.layer1 = nn.Conv2d(in_channels = in_channels, 
                            out_channels = 8, 
                            kernel_size = (3,3), 
                            padding = (1,1), 
                            stride = (1,1))
    self.maxpool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
    self.relu = nn.ReLU()
    self.bn = nn.BatchNorm2d(num_features = 8)
    self.layer2 = nn.Conv2d(in_channels = 8, 
                            out_channels = 16,
                            kernel_size = (3,3),
                            padding = (1,1),
                            stride = (1,1))
    self.fc = nn.Linear(in_features = 16*7*7, out_features = 256)
    self.outputs = nn.Linear(in_features = 256, out_features = num_classes)
    self.softmax = nn.Softmax()
  
  def forward(self, input_tensor):
    x = self.relu(self.layer1(input_tensor))
    x = self.bn(x)
    x = self.maxpool(x)
    x = self.relu(self.layer2(x))
    x = self.maxpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.relu(self.fc(x))
    x = self.softmax(self.outputs(x))
    return x


In [14]:
#Instantiating the class and test for the desired output
rand_input = torch.rand(size = (32, 1, 28,28))
model = CNN().to(device = device)
print(f">>>> The model'graph: {model}")
print(f">>>> the output shape: {model(rand_input).shape}")

>>>> The model'graph: CNN(
  (layer1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=784, out_features=256, bias=True)
  (outputs): Linear(in_features=256, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)
>>>> the output shape: torch.Size([32, 10])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [15]:
#fetching and pre-processing the data from the torchvission library

In [16]:
batch_size = 128
EPOCHS = 10
learning_rate = 1e-3
train_dfm = datasets.MNIST(root = "mnist_train/", train = True, transform = transforms.ToTensor(), download = True)
valid_dfm = datasets.MNIST(root = 'mnist_valid/', train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dfm, shuffle = True, batch_size = batch_size)
valid_loader = DataLoader(dataset = valid_dfm, shuffle = False, batch_size = batch_size)
x_train_batch, y_train_batch = next(iter(train_loader))
print(f">>>> x_train_batch_shape: {x_train_batch.shape}\ty_train_batch_shape: {y_train_batch.shape}")

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_train/MNIST/raw/train-images-idx3-ubyte.gz to mnist_train/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_train/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_train/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_train/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_train/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_train/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_train/MNIST/raw

Processing...
Done!


...

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_valid/MNIST/raw/train-images-idx3-ubyte.gz to mnist_valid/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_valid/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_valid/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_valid/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_valid/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_valid/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_valid/MNIST/raw

Processing...
Done!
>>>> x_train_batch_shape: torch.Size([128, 1, 28, 28])	y_train_batch_shape: torch.Size([128])


In [17]:
#Obtain the loss and optimizer objects
loss_obj = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)

In [18]:
#The training loop:
tic = time.time()
for epoch in range(EPOCHS):
  print(f">>>> training start for epoch: {epoch + 1}\n>>>> please wait while the model is training........................")
  for idx, (data, target) in enumerate(tqdm(train_loader)):
    data = data.to(device = device)
    target = target.to(device = device)
    #The forward pass 
    preds = model(data)
    train_loss = loss_obj(preds, target) #get the loss
    optimizer.zero_grad() #initialize the slopes to zero
    #backward pass
    train_loss.backward()
    optimizer.step() #gradient descent with adam optimizer

def __tracking__(loader, model):
  num_examples = 0
  num_correct = 0
  model.eval()
  #No need to re-compute the gradient during performance monitoring
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      preds = model(x)
      _, predictions = preds.max(1) #get the maximum prob in a softmax vector of probs (classes = 10)
      num_correct+=(predictions == y).sum() #Sum all the correct predictions
      num_examples+=predictions.size(0) #find total number of examples 
      acc = ((num_correct/num_examples)*100)
  model.train() #training the model
  return f">>>> at epoch: {epoch + 1}, loss = {train_loss:.4f}, train_accuracy: {acc:.4f}"
__tracking__(train_loader, model)

  




  0%|          | 1/469 [00:00<01:05,  7.11it/s]

>>>> training start for epoch: 1
>>>> please wait while the model is training........................


  0%|          | 2/469 [00:00<00:24, 19.06it/s]

>>>> training start for epoch: 2
>>>> please wait while the model is training........................


  0%|          | 2/469 [00:00<00:27, 16.85it/s]

...

>>>> training start for epoch: 9
>>>> please wait while the model is training........................


  0%|          | 2/469 [00:00<00:24, 18.80it/s]

>>>> training start for epoch: 10
>>>> please wait while the model is training........................


100%|██████████| 469/469 [00:25<00:00, 18.23it/s]


'>>>> at epoch: 10, loss = 1.4629, train_accuracy: 99.2467'